# MNIST classifier using CNN

## Authors:

Nicola Rocco de Luna Pedulla - nrlp


Renan de Freitas Lins - rfl3

Ricardo Rossiter Barioni - rrb

Victor Miranda de Melo - vmm

## Firstly, import the required dependencies

GPU is not being used

In [169]:
from DeepLearningPython35.network3 import load_data_shared, Network, ConvPoolLayer, FullyConnectedLayer, SoftmaxLayer, ReLU, size
import theano.tensor as T
import theano
import numpy as np

## From mnist_loader, gather the training, validation and test data
Each data is structured as a tuple of two elements, as follows: The first element of the tuple contains information about the image, while the second element contains the label of that tuple.

In the case of this dataset, the label tells the number (0 to 9) that the image represents.

In [170]:
training_data, validation_data, test_data = load_data_shared()
mini_batch_size = 10

# For smaller tests, it was created a subset of the training, validation and test data

In [171]:
def get_subset(data, l):
    
    x =  data[0].get_value(borrow=True)[:l]
    y = data[1].eval()[:l]

    shared_x = theano.shared( np.asarray(x, dtype=theano.config.floatX), borrow=True)
    shared_y = theano.shared( np.asarray(y, dtype=theano.config.floatX), borrow=True)
    return (shared_x, T.cast(shared_y, "int32"))

In [172]:

training_data_subset = get_subset(training_data, 2000)
test_data_subset = get_subset(test_data, 2000)
validation_data_subset = get_subset(validation_data, 2000)

# Perform network training

In [173]:
net = Network([
    ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28),
                  filter_shape=(20, 1, 5, 5),
                  poolsize=(2, 2),
                  activation_fn=ReLU),
    ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12),
                  filter_shape=(40, 20, 5, 5),
                  poolsize=(2, 2),
                  activation_fn=ReLU),
    FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
    SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 10, mini_batch_size, 0.03, validation_data, test_data, lmbda=0.1)

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 97.46%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.20%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 97.69%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.55%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 97.92%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.60%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

# Calculating the total accuracy of the test data

In [174]:
def get_accuracy(net, data):
    num_test_batches = int(size(data)/net.mini_batch_size)
    v = [net.test_mb_predictions(i) for i in range(num_test_batches)]
    pair = zip(np.concatenate(v),list(data[1].eval()))
    return np.mean([x == y for x,y in pair])

In [175]:
print('Accuracy', '(', '{0:.2f}'.format(100*get_accuracy(net,test_data))+'%',')')

Accuracy ( 98.07% )


# Calculating the accuracy for each of the possible labels (from 0 to 10)

In [176]:
test_data_y = test_data[1].eval()

num_test_batches = int(size(test_data)/net.mini_batch_size)
v = [net.test_mb_predictions(i) for i in range(num_test_batches)]

pair = list(zip(np.concatenate(v),list(test_data[1].eval())))

accuracies = [[x == y for x,y in pair if y == i ] for i in range(10)]

for i,val in enumerate(accuracies):
    print('label', i, ':', sum(val), '/', len(val), '(', '{0:.2f}'.format(100*np.mean(val))+ '%', ')')


label 0 : 967 / 980 ( 98.67% )
label 1 : 1115 / 1135 ( 98.24% )
label 2 : 1020 / 1032 ( 98.84% )
label 3 : 994 / 1010 ( 98.42% )
label 4 : 930 / 982 ( 94.70% )
label 5 : 888 / 892 ( 99.55% )
label 6 : 943 / 958 ( 98.43% )
label 7 : 1021 / 1028 ( 99.32% )
label 8 : 947 / 974 ( 97.23% )
label 9 : 982 / 1009 ( 97.32% )
